### Поиск организаций, имеющих лицензии
Реестр МО РФ получить с портала НСИ, извлечь из архива и поместитьв целевую папку as is;   
Выгрузку лицензий поместить as is, название файла должно начинаться со слова "лицензии" ("Лицензии"), формат .xlsx, лист "Лист 1";       
Данные по силовикам: as is, название файла должно начинаться со слова "силовики" ("Силовики"), формат .xlsx, лист "Лицензии+ФРМО";   
Справочник регионов тут: `C:/work/comparison_error_successful/comparison/reg_ref.csv`

In [1]:
%%time

import pandas as pd
import numpy as np
import os
import sys
import re
from datetime import date
pd.options.mode.chained_assignment = None  # default='warn'

CPU times: total: 0 ns
Wall time: 832 ms


In [2]:
%%time
# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов
dir_path = "C:/work/licenses"
ref_reg_path = "C:/work/comparison_error_successful/comparison/reg_ref.csv"
# ref_reg_path = "C:/work/licenses/reg_ref.csv"

for file_name in os.listdir(dir_path):
    if file_name.split("_")[0] == "1.2.643.5.1.13.13.11.1461":
        ref_path = dir_path + "/" + file_name
    elif file_name.split(" ")[0].split(".")[0].lower() == "силовики":
        army_path = dir_path + "/" + file_name
    elif file_name.split("_")[0].split(".")[0].lower() == "лицензии":
        lic_path = dir_path + "/" + file_name

df_ref = pd.read_excel(ref_path, sheet_name="Справочник", index_col=None, dtype="str")
df_lic = pd.read_excel(lic_path, sheet_name="Лист 1", index_col=None, dtype="str")
df_lic = df_lic.drop(
    index=range(0, df_lic[df_lic["Unnamed: 1"] == "Дата"].index[0])
).reset_index(drop=True)
df_lic.columns = df_lic.iloc[0]
df_lic = df_lic.drop(index=0).reset_index(drop=True)
df_army = pd.read_excel(
    army_path, sheet_name="Лицензии+ФРМО", index_col=None, dtype="str"
)

df_army_s = df_army[["№", "Дополнительный признак для фильтрации"]].rename(
    columns={"Дополнительный признак для фильтрации": "special_info"}
)

# df_lic_s = df_lic[["№", "ИНН", "ОГРН"]]
df_lic_s = df_lic.copy()

med_name_list = []
for name in df_ref["medicalSubjectName"].unique():
    if "едицинская" in name:
        med_name_list.append(name)
df_ref_s = df_ref[
    (df_ref["medicalSubjectName"].isin(med_name_list)) & (df_ref["deleteDate"].isnull())
][
    [
        "oid",
        "nameFull",
        "nameShort",
        "inn",
        "ogrn",
        "regionName",
        "moDeptName",
        "parentId",
        "organizationType",
    ]
]

df_lic_inn = df_lic_s.merge(df_ref_s, left_on="ИНН", right_on="inn", how="left")
df_lic_inn["origin"] = "ИНН"
df_lic_ogrn = df_lic_s.merge(df_ref_s, left_on="ОГРН", right_on="ogrn", how="left")
df_lic_ogrn["origin"] = "ОГРН"
df_lic_result = (
    pd.concat([df_lic_inn, df_lic_ogrn]).sort_values(by="№").reset_index(drop=True)
)
df_lic_result["oid"] = df_lic_result["oid"].fillna("не найден")

for index, row in df_lic_result.iterrows():
    if pd.isnull(row["oid"]):
        df_lic_result.loc[index, "origin"] = ""
for lic_num in df_lic_result["№"].unique():
    df_lic_0 = df_lic_result[df_lic_result["№"] == lic_num]
    for oid_num in df_lic_0["oid"].unique():
        df_lic_1 = df_lic_0[df_lic_0["oid"] == oid_num]
        if df_lic_1.shape[0] == 2:
            df_lic_result.loc[
                df_lic_1.index[0] : df_lic_1.index[-1], "origin"
            ] = "ИНН и ОГРН"
df_lic_result.drop_duplicates(inplace=True)
df_lic_result_a = df_lic_result.copy()
df_lic_result = df_lic_result_a.merge(df_army_s, left_on="№", right_on="№", how="left")
df_lic_result.drop(["inn", "ogrn"], axis=1, inplace=True)

# новый подход и корректный список регионов
df_lic_report = pd.DataFrame()
for l_num in df_lic_result["№"].unique().tolist():
    df_tmp = df_lic_result[df_lic_result["№"] == l_num]
    if df_tmp.shape[0] == 1:
        df_tmp["report"] = df_tmp["oid"]
    else:
        if (
            len(df_tmp["origin"].unique()) == 1
            and df_tmp["parentId"].isna().sum() == 1
            and set(df_tmp[~df_tmp["parentId"].isna()]["parentId"].unique()).issubset(
                df_tmp["oid"].unique()
            )
        ):
            df_tmp_0 = df_tmp[df_tmp["parentId"].isna()]
            df_tmp_0["report"] = df_tmp_0["oid"]
            df_tmp_1 = df_tmp[~df_tmp["parentId"].isna()]
            df_tmp_1["report"] = ""
            df_tmp = pd.concat([df_tmp_0, df_tmp_1])
        else:
            df_tmp["report"] = "требуется ручная проверка"
    df_lic_report = pd.concat([df_lic_report, df_tmp])
ref_reg_path = "C:/work/comparison_error_successful/comparison/reg_ref.csv"
df_reg_ref = pd.read_csv(ref_reg_path, sep=";")
df_lic_report = df_lic_report.merge(
    df_reg_ref, left_on="regionName", right_on="region_name_file", how="left"
)
df_lic_report = df_lic_report.drop("region_name_file", axis=1)
df_test = df_lic_report.copy()

# краткий выриант столбцов
# df_lic_report = df_lic_report[
#     [
#         "№",
#         "ИНН",
#         "ОГРН",
#         "oid",
#         "nameFull",
#         "nameShort",
#         "regionName",
#         "real_region_name",
#         "moDeptName",
#         "parentId",
#         "organizationType",
#         "origin",
#         "special_info",
#         "report",
#     ]
# ]

# полный вариант столбцов
df_lic_report = df_lic_report[
    [
        '№',
        'Дата',
        'Номер приказа',
        'Дата приказа',
        'Статус',
        'Наименование организации',
        'Полное наименование организации',
        'ИНН',
        'ОГРН',
        'Регион',
        'Юридический адрес',
        'Тип ЛО',
        'Кем выдана',
        'Вид деятельности',
        'oid',
        'nameFull',
        'nameShort',
        'regionName',
        'real_region_name',
        'moDeptName',
        'parentId',
        'organizationType',
        'origin',
        'special_info',
        'report',
    ]
]

today = date.today().strftime("%Y-%m-%d")

lic_inn_dict = {}
for lic_num in df_lic_inn["№"].unique():
    df_lic_l = df_lic_inn[df_lic_inn["№"] == lic_num]
    lic_inn_dict[lic_num] = df_lic_l["oid"].tolist()
df_inn = pd.DataFrame(columns=["№", "найдено по ИНН"])
for key, value in lic_inn_dict.items():
    df_inn.loc[len(df_inn.index)] = [key, value]

lic_ogrn_dict = {}
for lic_num in df_lic_ogrn["№"].unique():
    df_lic_l = df_lic_ogrn[df_lic_ogrn["№"] == lic_num]
    lic_ogrn_dict[lic_num] = df_lic_l["oid"].tolist()
df_ogrn = pd.DataFrame(columns=["№", "найдено по ОГРН"])
for key, value in lic_ogrn_dict.items():
    df_ogrn.loc[len(df_ogrn.index)] = [key, value]
df_lic_list = df_inn.merge(df_ogrn, left_on="№", right_on="№")

path_save = dir_path + "/" + "licenses_report_" + today + ".xlsx"
writer = pd.ExcelWriter(path_save, engine="xlsxwriter")
# df_lic_result.to_excel(writer, sheet_name="лицензии+НСИ", index=False)
df_lic_report.to_excel(writer, sheet_name="лицензии", index=False)
df_lic_list.to_excel(writer, sheet_name="oid списком", index=False)
writer.close()
# df_lic_result.head(5)

CPU times: total: 16.7 s
Wall time: 1min 23s


In [3]:
df_lic_report.head(3)

,№,Дата,Номер приказа,Дата приказа,Статус,Наименование организации,Полное наименование организации,ИНН,ОГРН,Регион,...,nameFull,nameShort,regionName,real_region_name,moDeptName,parentId,organizationType,origin,special_info,report
0,Л041-00110-01/00357877,12.11.2020,1032,27.12.2023,Действует,ГБУЗ РА АРКБ,Государственное бюджетное учреждение здравоохр...,0105025965,1020100703842,Республика Адыгея,...,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,ГБУЗРА АРКБ,Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Органы исполнительной власти субъектов Российс...,NaN,1,ИНН и ОГРН,NaN,1.2.643.5.1.13.13.12.2.1.32
1,Л041-00110-01/00361707,23.09.2019,6929,29.07.2022,Действует,"ФГБОУ ВО ""МГТУ""",Федеральное государственное бюджетное образова...,0105014177,1020100698595,Республика Адыгея,...,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,"ФГБОУ ВО ""МГТУ"", МГТУ, МАЙКОПСКИЙ ГОСУДАРСТВЕН...",Республика Адыгея (Адыгея),Республика Адыгея (Адыгея),Министерство образования и науки Российской Фе...,NaN,1,ИНН и ОГРН,NaN,1.2.643.5.1.13.13.12.3.1.3434
2,Л041-00110-01/00571612,12.12.2017,23-Пр/24,26.03.2024,Действует,МСЧ МВД по Республике Адыгея,Федеральное казенное учреждение здравоохранен...,0105047430,1060105011361,Республика Адыгея,...,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,"ФКУЗ ""МСЧ МВД РОССИИ ПО РЕСПУБЛИКЕ АДЫГЕЯ""",Республика Адыгея,Республика Адыгея (Адыгея),Министерство внутренних дел Российской Федерации,NaN,1,ИНН и ОГРН,МСЧ / поликлиника / госпиталь МВД,1.2.643.5.1.13.13.12.2.1.15354


In [43]:
rzn_path = r"C:\Users\Admin\Desktop\поисковик им. Антона\test\файл_РЗН.xlsx"
df_rzn = pd.read_excel(rzn_path, sheet_name="рзн", index_col=None, dtype="str")

In [44]:
df_rzn.head(1)

,Наименование организации,Полное наименование организации,ИНН,ОГРН,Регион организации,Юридический адрес,Регион объекта,Адрес объекта,Тип населенного пункта,Код ФИАС,Тип объекта,№,Дата,Статус,Работы/услуги,Категория риска РОМ (фарм.розн.),Категория риска РОМ (фарм.опт)
0,"""Медицинский центр Дар""","Общество с ограниченной ответственностью ""Меди...",8602310766,1238600005115,Ханты-Мансийский автономный округ - Югра,"628426, Россия, Ханты-Мансийский автономный ок...",Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",NaN,f60cb57e-5248-4b9e-bd1d-603c1ef1b475,NaN,Л041-01350-86/00741812,23.10.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN


In [69]:
df_rzn["address_found"] = ""
df_rzn["oid_found"] = ""
n = 0
for index, row in df_rzn.iterrows():
    if n < 2:
        ratio = 0
        rzn_add_list = str(row["Адрес объекта"]).replace(".", "").replace(",", "").replace("-", "").lower().split(" ")
        for i, r in df_spadd.iterrows():
            sp_add_list = str(r["bld_address"]).replace(".", "").replace(",", "").replace("-", "").lower().split(" ")
            lst = [x for x in rzn_add_list if x in sp_add_list]
            ratio_tmp = len(lst)*200/(max(len(rzn_add_list), len(sp_add_list)))
            # ratio_tmp = len(lst)
            if ratio_tmp > ratio:
                oid = r["sp_oid"]
                add = r["bld_address"]
                ratio = ratio_tmp
        df_rzn.at[index, "address_found"] = add
        df_rzn.at[index, "oid_found"] = oid
        n+=1
        print(f"сделано {n} из {df_rzn.shape[0]}")
    else:
        break

сделано 1 из 12182
сделано 2 из 12182


In [70]:
df_rzn

,Наименование организации,Полное наименование организации,ИНН,ОГРН,Регион организации,Юридический адрес,Регион объекта,Адрес объекта,Тип населенного пункта,Код ФИАС,Тип объекта,№,Дата,Статус,Работы/услуги,Категория риска РОМ (фарм.розн.),Категория риска РОМ (фарм.опт),address_found,oid_found
0,"""Медицинский центр Дар""","Общество с ограниченной ответственностью ""Меди...",8602310766,1238600005115,Ханты-Мансийский автономный округ - Югра,"628426, Россия, Ханты-Мансийский автономный ок...",Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",NaN,f60cb57e-5248-4b9e-bd1d-603c1ef1b475,NaN,Л041-01350-86/00741812,23.10.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,Ханты-Мансийский Автономный округ - Югра Автон...,1.2.643.5.1.13.13.12.2.86.8956.0.124135
1,МЦ СургутАвиаМед,Общество с ограниченной ответственностью Медиц...,8602309680,1238600001672,Ханты-Мансийский автономный округ - Югра,"628422, Россия, Ханты-Мансийский автономный ок...",Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",NaN,6c2aadb8-4412-4ed5-9082-bd1131a3945b,NaN,Л041-01350-86/00666954,28.07.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,Ханты-Мансийский Автономный округ - Югра Автон...,1.2.643.5.1.13.13.12.2.86.86896.0.664617
2,"""МЕДСПРАВКА""","Общество с ограниченной ответственностью ""МЕДС...",2634110663,1222600010005,Ставропольский край,"355017, Россия, Ставропольский край, городской...",Ставропольский край,"355017, Ставропольский край, г Ставрополь, ул ...",NaN,0fc8937c-bb24-4a4f-b4ab-7cd0cd101747,NaN,Л041-01197-26/00639943,03.02.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,,
3,"""Предгорная районная больница""",государственное бюджетное учреждение здравоохр...,2618010703,1022600962240,Ставропольский край,"357350, Россия, Ставропольский край, Предгорны...",Ставропольский край,"357350, Ставропольский край, Предгорный р-н, с...",станица,58e93788-5cd4-4962-b334-378ffae40f96,NaN,Л041-01197-26/00574633,24.12.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,,
4,"""Благодарненская районная больница""",государственное бюджетное учреждение здравоохр...,2605010625,1022602421863,Ставропольский край,"356420, Россия, Ставропольский край, Благодарн...",Ставропольский край,"356420, Ставропольский край, Благодарненский р...",NaN,b277c385-a390-4ac4-a85d-0162fee81c99,NaN,Л041-01197-26/00574277,28.01.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12177,УфИЦ РАН,Федеральное государственное бюджетное научное ...,0274064870,1030204207582,Республика Башкортостан,"450098, Россия, Республика Башкортостан, Уфа, ...",Республика Башкортостан,"450054, Республика Башкортостан, г. Уфа, пр. О...",город,faf12493-d842-40bc-968a-938e86dd8d70,NaN,Л041-00110-02/00340505,01.11.2018,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,,
12178,федеральное казенное учреждение здравоохранени...,федеральное казенное учреждение здравоохранени...,0275006624,1020202776109,Республика Башкортостан,"450015, Россия, Республика Башкортостан, г. Уф...",Республика Башкортостан,"450015, Республика Башкортостан, г. Уфа, ул. К...",город,236bd30d-7d8d-4baa-a322-44806a79b34a,NaN,Л041-00110-02/00323564,02.12.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,,
12179,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,Федеральное казенное учреждение здравоохранен...,0105047430,1060105011361,Республика Адыгея,"385006, Россия, Республика Адыгея, г. Майкоп, ...",Республика Адыгея,"385006, Республика Адыгея, г. Майкоп, ул. Стро...",город,105aa593-0e4b-4769-8c09-ab9ff4cb12fe,NaN,Л041-00110-01/00571612,12.12.2017,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,,
12180,"Фгбоу во ""Мгту"", мгту, майкопский государствен...",Федеральное государственное бюджетное образова...,0105014177,

In [17]:
spadd_path = r"C:\work\scripts\справочники\1.2.643.5.1.13.13.99.2.1123_1.7.xlsx"
df_spadd = pd.read_excel(spadr_path, sheet_name="Справочник", index_col=None)

In [20]:
df_spadd.head(2)

,id,mo_region_name,mo_oid,mo_name_short,sp_oid,sp_name,fp_oid,fp_name,fp_type_name,bld_uuid,bld_id,bld_name,bld_address
0,ID записи справочника,Субъект РФ,OID медицинской организации,Краткое наименование медицинской организации,OID структурного подразделения,Наименование структурного подразделения,OID отделения/кабинета,Наименование отделения/кабинета,Наименование типа отделения/кабинета,GUID здания,ID здания,Наименование здания,Адрес здания
1,1,Свердловская область,1.2.643.5.1.13.13.12.2.66.22754,"ООО ""МКС ""БЕЛАЯ ЛАДЬЯ""",1.2.643.5.1.13.13.12.2.66.22754.0.460317,Фитнес центр,1.2.643.5.1.13.13.12.2.66.22754.0.460317.454771,NaN,Кабинет лечебной физкультуры,029075ac-3ca5-4bf7-8ef8-aa7d08307d90,163361,Фитнес центр,Свердловская Область г Екатеринбург ул Комсомо...


In [51]:
str_rzn = "628426, Ханты-Мансийский автономный округ - Югра, г. Сургут, проспект Мира, д. 55, этаж 1"
str_add = "Ханты-Мансийский Автономный округ - Югра Автономный округ г Сургут ул Губкина 1"
lst_rzn = str_rzn.replace(".", "").replace(",", "").replace("-", "").lower().split(" ")
lst_add = str_add.replace(".", "").replace(",", "").replace("-", "").lower().split(" ")


In [52]:
lst = [x for x in lst_rzn if x in lst_add]
lst

['хантымансийский', 'автономный', 'округ', '', 'югра', 'г', 'сургут', '1']

In [53]:
ratio = len(lst)*200/(len(lst_rzn)+len(lst_add))
ratio = len(lst)
ratio

8

***
СЭМД-РЭМД

In [3]:
import pandas as pd
import numpy as np
import os
import sys
from datetime import date
pd.options.mode.chained_assignment = None  # default='warn'

from difflib import SequenceMatcher

In [54]:
def ind(df):
    df.insert(0,"ID", "")
    for i in range (df.shape[0]):
        df.at[i, "ID"] = i
    return df

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

rzn_path = r"C:\work\Медкнижки\semd_remd\sr\объекты_медосмотры_предварительные_для_МЗ_02_04_2024.xlsx"
s194_path = r"C:\work\Медкнижки\semd_remd\sr\РЭМД. 194 04.04.xlsx"
s230_path = r"C:\work\Медкнижки\semd_remd\sr\РЭМД. 230 04.04.xlsx"

df_rzn = pd.read_excel(rzn_path, sheet_name="rzn", index_col=None)
df_s194 = pd.read_excel(s194_path, sheet_name="194", index_col=None).fillna(0)
df_s230 = pd.read_excel(s230_path, sheet_name="230", index_col=None).fillna(0)

df_rzn = ind(df_rzn)
df_s194 = ind(df_s194)
df_s230 = ind(df_s230)

reg_ref_path = r"C:\work\Медкнижки\semd_remd\semd_reg_ref.csv"
df_reg_ref = pd.read_csv(reg_ref_path, sep=";")

In [4]:
df_rzn = df_rzn.merge(df_reg_ref, on="Регион объекта")

In [22]:
%%time

df_rzn_a = df_rzn[["ID", "Регион объекта", "Адрес объекта", "region_semd"]]
df_s230_a = df_s230[["ID", "Субъект РФ", "Адрес СП МО"]]
df_s194_a = df_s194[["ID", "Субъект РФ", "Адрес СП МО"]]
df_rzn_a["ID_194"] = ""
df_rzn_a["ID_230"] = ""
df_rzn_a["address_194"] = ""
df_rzn_a["address_230"] = ""
df_rzn_a["r_194"] = ""
df_rzn_a["r_230"] = ""
n = 0
for index, row in df_rzn_a.iterrows():
    df_s194_tmp = df_s194_a[df_s194_a["Субъект РФ"] == row["region_semd"]]
    df_s230_tmp = df_s230_a[df_s230_a["Субъект РФ"] == row["region_semd"]]
    rzn_add_list = str(row["Адрес объекта"]).replace(".", "").replace(",", "").replace("-", "").lower().split(" ")
    rzn_add_list = [x for x in rzn_add_list if len(x) > 4]
    ratio_194 = 0
    ratio_230 = 0
    if df_s194_tmp.shape[0] > 0:
        for i, r in df_s194_tmp.iterrows():
            if r["Адрес СП МО"] != 0 and r["Адрес СП МО"] != "Не указано":
                s_194_add_list = (str(row["region_semd"]) + " " + str(r["Адрес СП МО"])).replace(".", "").replace(",", "").lower().split(" ")
                s_194_add_list = [x for x in s_194_add_list if len(x) > 4]
                lst = [x for x in rzn_add_list if x in s_194_add_list]
                ratio_tmp_194 = len(lst)/min(len(rzn_add_list), len(s_194_add_list))
                if ratio_tmp_194 > ratio_194:
                        ID_194 = r["ID"]
                        add_194 = r["Адрес СП МО"]
                        ratio_194 = ratio_tmp_194
            else:
                ID_194 = ""
                add_194 = ""
                ratio_194 = 0
    if df_s230_tmp.shape[0] > 0:
        for i, r in df_s230_tmp.iterrows():
            if r["Адрес СП МО"] != 0 and r["Адрес СП МО"] != "Не указано":
                s_230_add_list = (str(row["region_semd"]) + " " + str(r["Адрес СП МО"])).replace(".", "").replace(",", "").lower().split(" ")
                s_230_add_list = [x for x in s_230_add_list if len(x) > 4]
                lst = [x for x in rzn_add_list if x in s_230_add_list]
                ratio_tmp_230 = len(lst)/min(len(rzn_add_list), len(s_230_add_list))
                if ratio_tmp_230 > ratio_230:
                        ID_230 = r["ID"]
                        add_230 = r["Адрес СП МО"]
                        ratio_230 = ratio_tmp_230
            else:
                ID_230 = ""
                add_230 = ""
                ratio_230 = 0
    n+=1
    df_rzn_a.at[index, "ID_194"] = ID_194
    df_rzn_a.at[index, "ID_230"] = ID_230
    df_rzn_a.at[index, "address_194"] = add_194
    df_rzn_a.at[index, "address_230"] = add_230
    df_rzn_a.at[index, "r_194"] = round(ratio_194, 1)
    df_rzn_a.at[index, "r_230"] = round(ratio_230, 1)
    ID_194 = ""
    ID_230 = ""
    add_194 = ""
    add_230 = ""
    print(f"сделано {n} из {df_rzn_a.shape[0]}: index: {index}, ratio_194: {ratio_194}, ratio_230: {ratio_230}")
    # break

сделано 1 из 12182: index: 0, ratio_194: 0.5, ratio_230: 0
сделано 2 из 12182: index: 1, ratio_194: 0.5, ratio_230: 0
сделано 3 из 12182: index: 2, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 4 из 12182: index: 3, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 5 из 12182: index: 4, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 6 из 12182: index: 5, ratio_194: 0.4, ratio_230: 0
сделано 7 из 12182: index: 6, ratio_194: 0.4, ratio_230: 0
сделано 8 из 12182: index: 7, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 9 из 12182: index: 8, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 10 из 12182: index: 9, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 11 из 12182: index: 10, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 12 из 12182: index: 11, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 13 из 12182: index: 12, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 14 из 12182: index: 13, ratio_194: 0.3333333333333333, ratio_230: 0
сделано 15 из 1

In [55]:
df_rzn

,ID,Наименование организации,Полное наименование организации,ИНН,ОГРН,Регион организации,Юридический адрес,Регион объекта,Адрес объекта,Тип населенного пункта,Код ФИАС,Тип объекта,№,Дата,Статус,Работы/услуги,Категория риска РОМ (фарм.розн.),Категория риска РОМ (фарм.опт)
0,0,"""Медицинский центр Дар""","Общество с ограниченной ответственностью ""Меди...",8602310766,1238600005115,Ханты-Мансийский автономный округ - Югра,"628426, Россия, Ханты-Мансийский автономный ок...",Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",NaN,f60cb57e-5248-4b9e-bd1d-603c1ef1b475,NaN,Л041-01350-86/00741812,23.10.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN
1,1,МЦ СургутАвиаМед,Общество с ограниченной ответственностью Медиц...,8602309680,1238600001672,Ханты-Мансийский автономный округ - Югра,"628422, Россия, Ханты-Мансийский автономный ок...",Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",NaN,6c2aadb8-4412-4ed5-9082-bd1131a3945b,NaN,Л041-01350-86/00666954,28.07.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN
2,2,"""МЕДСПРАВКА""","Общество с ограниченной ответственностью ""МЕДС...",2634110663,1222600010005,Ставропольский край,"355017, Россия, Ставропольский край, городской...",Ставропольский край,"355017, Ставропольский край, г Ставрополь, ул ...",NaN,0fc8937c-bb24-4a4f-b4ab-7cd0cd101747,NaN,Л041-01197-26/00639943,03.02.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN
3,3,"""Предгорная районная больница""",государственное бюджетное учреждение здравоохр...,2618010703,1022600962240,Ставропольский край,"357350, Россия, Ставропольский край, Предгорны...",Ставропольский край,"357350, Ставропольский край, Предгорный р-н, с...",станица,58e93788-5cd4-4962-b334-378ffae40f96,NaN,Л041-01197-26/00574633,24.12.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN
4,4,"""Благодарненская районная больница""",государственное бюджетное учреждение здравоохр...,2605010625,1022602421863,Ставропольский край,"356420, Россия, Ставропольский край, Благодарн...",Ставропольский край,"356420, Ставропольский край, Благодарненский р...",NaN,b277c385-a390-4ac4-a85d-0162fee81c99,NaN,Л041-01197-26/00574277,28.01.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12177,12177,УфИЦ РАН,Федеральное государственное бюджетное научное ...,274064870,1030204207582,Республика Башкортостан,"450098, Россия, Республика Башкортостан, Уфа, ...",Республика Башкортостан,"450054, Республика Башкортостан, г. Уфа, пр. О...",город,faf12493-d842-40bc-968a-938e86dd8d70,NaN,Л041-00110-02/00340505,01.11.2018,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN
12178,12178,федеральное казенное учреждение здравоохранени...,федеральное казенное учреждение здравоохранени...,275006624,1020202776109,Республика Башкортостан,"450015, Россия, Республика Башкортостан, г. Уф...",Республика Башкортостан,"450015, Республика Башкортостан, г. Уфа, ул. К...",город,236bd30d-7d8d-4baa-a322-44806a79b34a,NaN,Л041-00110-02/00323564,02.12.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN
12179,12179,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,Федеральное казенное учреждение здравоохранен...,105047430,1060105011361,Республика Адыгея,"385006, Россия, Республика Адыгея, г. Майкоп, ...",Республика Адыгея,"385006, Республика Адыгея, г. Майкоп, ул. Стро...",город,105aa593-0e4b-4769-8c09-ab9ff4cb12fe,NaN,Л041-00110-01/00571612,12.12.2017,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN
12180,12180,"Фгбоу во ""Мгту"", мгту, майкопский государствен...",Федеральное государственное бюджетное образова...,105014177,1020100698595,Республика Адыгея,"385000, Россия, Республика Адыгея, г. Майкоп, ...",Республика Адыгея,"385000, Республика Адыгея, г. Майкоп, Комсомол...",город,da529930-8597-46b3-b1ca-a49b

In [42]:
%%time

df_rzn_a = df_rzn[["ID", "Регион объекта", "Адрес объекта", "region_semd"]]
df_s230_a = df_s230[["ID", "Субъект РФ", "Адрес СП МО"]]
df_s194_a = df_s194[["ID", "Субъект РФ", "Адрес СП МО"]]
df_rzn_a["ID_194"] = ""
df_rzn_a["ID_230"] = ""
df_rzn_a["address_194"] = ""
df_rzn_a["address_230"] = ""
df_rzn_a["r_194"] = ""
df_rzn_a["r_230"] = ""
n = 0
for index, row in df_rzn_a.iterrows():
    df_s194_tmp = df_s194_a[df_s194_a["Субъект РФ"] == row["region_semd"]]
    df_s230_tmp = df_s230_a[df_s230_a["Субъект РФ"] == row["region_semd"]]
    ratio_194 = 0
    ratio_230 = 0
    if df_s194_tmp.shape[0] > 0:
        for i, r in df_s194_tmp.iterrows():
            # if r["Адрес СП МО"] != 0 and r["Адрес СП МО"] != "Не указано":
            ratio_tmp_194 = similar(str(row["Адрес объекта"]), (str(row["region_semd"]) + " " + str(r["Адрес СП МО"])))
            if ratio_tmp_194 > ratio_194:
                    ID_194 = r["ID"]
                    add_194 = r["Адрес СП МО"]
                    ratio_194 = ratio_tmp_194
            # else:
            #     ID_194 = ""
            #     add_194 = ""
            #     ratio_194 = 0
    if df_s230_tmp.shape[0] > 0:
        for i, r in df_s230_tmp.iterrows():
            # if r["Адрес СП МО"] != 0 and r["Адрес СП МО"] != "Не указано":
            ratio_tmp_230 = similar(str(row["Адрес объекта"]), (str(row["region_semd"]) + " " + str(r["Адрес СП МО"])))
            if ratio_tmp_230 > ratio_230:
                    ID_230 = r["ID"]
                    add_230 = r["Адрес СП МО"]
                    ratio_230 = ratio_tmp_230
            # else:
            #     ID_230 = ""
            #     add_230 = ""
            #     ratio_230 = 0
    n+=1
    df_rzn_a.at[index, "ID_194"] = ID_194
    df_rzn_a.at[index, "ID_230"] = ID_230
    df_rzn_a.at[index, "address_194"] = add_194
    df_rzn_a.at[index, "address_230"] = add_230
    df_rzn_a.at[index, "r_194"] = round(ratio_194, 1)
    df_rzn_a.at[index, "r_230"] = round(ratio_230, 1)
    ID_194 = ""
    ID_230 = ""
    add_194 = ""
    add_230 = ""
    print(f"сделано {n} из {df_rzn_a.shape[0]}: index: {index}, ratio_194: {ratio_194}, ratio_230: {ratio_230}")
    # break

сделано 1 из 12182: index: 0, ratio_194: 0.7411764705882353, ratio_230: 0.6285714285714286
сделано 2 из 12182: index: 1, ratio_194: 0.8074534161490683, ratio_230: 0.6616541353383458
сделано 3 из 12182: index: 2, ratio_194: 0.7083333333333334, ratio_230: 0.6097560975609756
сделано 4 из 12182: index: 3, ratio_194: 0.7978723404255319, ratio_230: 0.6043956043956044
сделано 5 из 12182: index: 4, ratio_194: 0.8354430379746836, ratio_230: 0.7
сделано 6 из 12182: index: 5, ratio_194: 0.8590604026845637, ratio_230: 0.6890756302521008
сделано 7 из 12182: index: 6, ratio_194: 0.87248322147651, ratio_230: 0.7058823529411765
сделано 8 из 12182: index: 7, ratio_194: 0.8461538461538461, ratio_230: 0.64
сделано 9 из 12182: index: 8, ratio_194: 0.8491620111731844, ratio_230: 0.6588235294117647
сделано 10 из 12182: index: 9, ratio_194: 0.7975460122699386, ratio_230: 0.6956521739130435
сделано 11 из 12182: index: 10, ratio_194: 0.7610619469026548, ratio_230: 0.5073170731707317
сделано 12 из 12182: index:

In [43]:
df_rzn_a

,ID,Регион объекта,Адрес объекта,region_semd,ID_194,ID_230,address_194,address_230,r_194,r_230
0,0,Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",Ханты-Мансийский автономный округ - Югра,5279,168,"628404, г Сургут, пр-кт Набережный, д.41",Не указано,0.7,0.6
1,1,Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",Ханты-Мансийский автономный округ - Югра,5324,168,"628418, г Сургут, ул Островского, д.15",Не указано,0.8,0.7
2,281,Ханты-Мансийский автономный округ - Югра,"628609, Ханты-Мансийский автономный округ - Юг...",Ханты-Мансийский автономный округ - Югра,5344,168,"628600, г Нижневартовск, ул Мира, д.63",Не указано,0.7,0.6
3,282,Ханты-Мансийский автономный округ - Югра,"628250, Ханты-Мансийский автономный округ - Юг...",Ханты-Мансийский автономный округ - Югра,5310,167,"628250, пгт Пионерский, ул Советская, д.65","628307, г Нефтеюганск, мкр. 8-й, д.8",0.8,0.6
4,283,Ханты-Мансийский автономный округ - Югра,"628464, Ханты-Мансийский автономный округ - Ю...",Ханты-Мансийский автономный округ - Югра,5314,167,"628462, г.о. Радужный, мкр 6, д.29","628307, г Нефтеюганск, мкр. 8-й, д.8",0.8,0.7
...,...,...,...,...,...,...,...,...,...,...
12177,11574,Оренбургская область,"460000, Оренбургская область, г. Оренбург, ули...",Оренбургская область,2715,,"460035, г Оренбург, ул 1 Мая, д.61",,0.7,0
12178,11144,Байконур,"468320, Байконур, г. Байконур, ул. Набережная,...",г. Байконур,678,,"468320, ул Набережная",,0.6,0
12179,11145,Байконур,"468320, Байконур, г. Байконур, ул. Жанкожа-Бат...",г. Байконур,677,,"468320, ул Жанкожа-Батыра, д.2",,0.7,0
12180,11146,Байконур,"468320, Байконур, г. Байконур, 6 мкр., ул. Бар...",г. Байконур,677,,"468320, ул Жанкожа-Батыра, д.2",,0.5,0


In [26]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [41]:
similar(str_0, str_3)

0.7073170731707317

***
ОКТМО и ОКАТО

In [80]:
def okt (df):
    """функция расшифровывает справочники ОКАТО и ОКТМО"""
    import pymorphy3
    morph = pymorphy3.MorphAnalyzer()

    ter = 0
    kod1 = 0
    kod2 = 0
    kod3 = 0
    e0 = ""
    e1 = ""
    e2 = ""
    e3 = ""
    ep = ""

    okt_list = []
    for index, row in df.iterrows():
        if row["TER"] != ter:
            e0 = " ".join([morph.parse(x)[0][2].title() for x in row["NAME1"].split(" ")][2:])
            ter = row["TER"]
        if row["KOD1"] != kod1 and row["KOD1"] % 100 != 0:
            e1 = row["NAME1"]
            e2 = ""
            e3 = ""
            ep = ""
            kod1 = row["KOD1"]
        if row["KOD2"] != kod2 and row["KOD2"] % 100 != 0:
            e2 = row["NAME1"]
            e3 = ""
            ep = ""
            kod2 = row["KOD2"]
        if row["KOD3"] != kod3 and row["KOD3"] != 0:
            e3 = " ".join(row["NAME1"].split(" ")[1:])
            ep = row["NAME1"].split(" ")[0]
            kod3 = row["KOD3"]
        else:
            kod3 = row["KOD3"]
        okt_list = okt_list + [[e0, e1, e2, ep, e3]]
    df_t = pd.DataFrame(okt_list, columns =["level_0", "level_1", "level_2", "prefix", "settelment"])
    df_t = df_t.drop_duplicates().reset_index(drop = True)
    return df_t

In [12]:
%%time

oktmo_path = r"C:\Users\Admin\Desktop\ОКТМО.xlsx"
df_oktmo = pd.read_excel(oktmo_path, sheet_name="ОКТМО", index_col=None)

CPU times: total: 5.33 s
Wall time: 16.6 s


In [134]:
df_oktmo

,TER,KOD1,KOD2,KOD3,KC,RAZDEL,NAME1,Centrum,NomDescr,NomAkt,Status,DateUtv,DateVved
0,0,0,0,0,0,1,Раздел 1. Муниципальные образования субъектов ...,NaN,NaN,0,0,2013-06-14,2014-01-01
1,0,0,0,0,0,2,"Раздел 2. Населенные пункты, входящие в состав...",NaN,NaN,0,0,2013-06-14,2014-01-01
2,1,0,0,0,2,1,Муниципальные образования Алтайского края,NaN,NaN,0,0,2013-06-14,2014-01-01
3,1,0,0,0,2,2,"Населенные пункты, входящие в состав муниципал...",NaN,NaN,0,0,2013-06-14,2014-01-01
4,1,500,0,0,6,1,Муниципальные округа Алтайского края,NaN,NaN,493,3,2021-08-12,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195447,99,630,440,116,2,2,ст Лумку-Корань,NaN,NaN,0,0,2013-06-14,2014-01-01
195448,99,700,0,0,4,1,Городские округа Еврейской автономной области/,NaN,NaN,0,0,2013-06-14,2014-01-01
195449,99,701,0,0,9,1,город Биробиджан,г Биробиджан,NaN,0,0,2013-06-14,2014-01-01
195450,99,701,0,0,9,2,"Населенные пункты, входящие в состав городског...",NaN,NaN,0,0,2013-06-14,2014-01-01


In [169]:
ter_list = [x for x in df_oktmo["TER"].unique() if x not in [0]]
kod1_list = [x for x in df_oktmo["KOD1"].unique() if x not in [0, 300, 400, 500, 600, 700, 800, 900]]
kod2_list = [x for x in df_oktmo["KOD2"].unique() if x not in [100, 300, 400, 700]]
df_oktmo[(df_oktmo["TER"] == ter_list[13]) & (df_oktmo["KOD1"] == kod1_list[0]) & (df_oktmo["KOD2"] == kod2_list[0])]

,TER,KOD1,KOD2,KOD3,KC,RAZDEL,NAME1,Centrum,NomDescr,NomAkt,Status,DateUtv,DateVved
27925,19,512,0,0,4,1,Вашкинский муниципальный округ,с Липин Бор,NaN,637,3,2023-08-15,2024-01-01
27926,19,512,0,0,4,2,"Населенные пункты, входящие в состав Вашкинско...",NaN,NaN,637,3,2023-08-15,2024-01-01
27927,19,512,0,101,3,2,с Липин Бор,NaN,NaN,637,3,2023-08-15,2024-01-01
27928,19,512,0,103,5,2,д Аксентьево,NaN,NaN,637,3,2023-08-15,2024-01-01
27929,19,512,0,105,7,2,д Андреевская,NaN,NaN,637,3,2023-08-15,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28119,19,512,0,485,4,2,п Первомайский,NaN,NaN,637,3,2023-08-15,2024-01-01
28120,19,512,0,487,6,2,с Вашкозерки,NaN,NaN,637,3,2023-08-15,2024-01-01
28121,19,512,0,489,8,2,с Киуй,NaN,NaN,637,3,2023-08-15,2024-01-01
28122,19,512,0,491,0,2,с Троицкое,NaN,NaN,637,3,2023-08-15,2024-01-01


In [81]:
%%time

df_oktmo_parse = okt(df_oktmo)
df_oktmo_parse

CPU times: total: 38.3 s
Wall time: 3min 15s


,level_0,level_1,level_2,prefix,settelment
0,,,,,
1,Алтайский Край,,,,
2,Алтайский Край,Залесовский муниципальный округ,,,
3,Алтайский Край,Залесовский муниципальный округ,,с,Залесово
4,Алтайский Край,Залесовский муниципальный округ,,с,Большой Калтай
...,...,...,...,...,...
174798,Еврейский Автономный Область,Смидовичский муниципальный район,Волочаевское,с,Волочаевка-1
174799,Еврейский Автономный Область,Смидовичский муниципальный район,Волочаевское,ст,Ольгохта
174800,Еврейский Автономный Область,Смидовичский муниципальный район,Волочаевское,ст,Лумку-Корань
174801,Еврейский Автономный Область,город Биробиджан,,,


In [49]:
# OKATO
# okato_path = r"C:\Users\Admin\Desktop\ОКАТО.xlsx"
# df_okato = pd.read_excel(oktmo_path, sheet_name="ОКТМО", index_col=None)

In [50]:
# df_okato_parse = okt(df_okato)
# df_okato_parse

In [2]:
df_adr_rzn = pd.DataFrame(df_rzn["Адрес объекта"])
df_adr_test = df_adr_rzn.loc[[1, 300, 500, 1000, 3000, 5000]]
df_adr_test

NameError: name 'df_rzn' is not defined

In [32]:
# df_adr_test["a"] = df_adr_test["Адрес объекта"].apply(lambda x: str(x).lower().replace(r'\W', '', regex=True).replace('ё', 'е'))
# df_adr_test["a"] = df_adr_test["Адрес объекта"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
# df_adr_test

In [85]:
set(df_oktmo_parse["level_0"].tolist())

{'',
 'Алтайский Край',
 'Амурский Область',
 'Архангельский Область',
 'Астраханский Область',
 'Белгородский Область',
 'Брянский Область',
 'Владимирский Область',
 'Волгоградский Область',
 'Вологодский Область',
 'Воронежский Область',
 'Город Москва (Столица Российский Федерация Город Федеральный Значения)',
 'Город Федеральный Значение Санкт-Петербург',
 'Город Федеральный Значение Севастополь',
 'Донецкий Народный Республика',
 'Еврейский Автономный Область',
 'Забайкальский Край',
 'Запорожский Область',
 'Ивановский Область',
 'Иркутский Область',
 'Кабардино-Балкарский Республика',
 'Калининградский Область',
 'Калужский Область',
 'Камчатский Край',
 'Карачаево-Черкесский Республика',
 'Кемеровский Область - Кузбасс',
 'Кировский Область',
 'Костромской Область',
 'Краснодарский Край',
 'Красноярский Край',
 'Курганский Область',
 'Курский Область',
 'Ленинградский Область',
 'Липецкий Область',
 'Луганский Народный Республика',
 'Магаданский Область',
 'Московский Область'

In [127]:
l0_list = set(df_oktmo_parse["level_0"].tolist())
l1_list = set(df_oktmo_parse["level_1"].tolist())
l2_list = set(df_oktmo_parse["level_2"].tolist())
full_address_list = []
for address in df_adr_test["Адрес объекта"].tolist():
    address = str(address).lower().replace('ё', 'е').replace(',', '').replace('.', ' ').replace(' - ', ' ')
    ad_list = address.split(" ")
    ad_list = [morph.parse(x)[0][2].title() for x in ad_list if len(x) > 2]
    address_list = []
    for w in ad_list:
        print(w)
        w_lst = []
        for element in l0_list:
            if element.find(w) != -1:
                w_lst.append(element)
        if len(w_lst) < 3:
            w_lst = w_lst
            col_name = "level_0"
        else:
            w_lst = []
        print(w_lst, col_name)
        if len(w_lst) == 0:
            print(w)
            w_lst = []
            for element in l1_list:
                if element.find(w) != -1:
                    w_lst.append(element)
            if len(w_lst) < 3:
                w_lst = w_lst
                col_name = "level_1"
            else:
                w_lst = []
            print(w_lst, col_name)
        if len(w_lst) != 0 and w not in ["Автономный", "Округ", "Пер"]:
            address_list.append([address, col_name, w, w_lst])
    full_address_list.append(address_list)
    break

628426
[] level_0
628426
[] level_1
Ханты-Мансийский
[] level_0
Ханты-Мансийский
['Ханты-Мансийский муниципальный район'] level_1
Автономный
['Еврейский Автономный Область', 'Чукотский Автономный Округ'] level_0
Округ
['Чукотский Автономный Округ'] level_0
Югра
[] level_0
Югра
[] level_1
Сургут
[] level_0
Сургут
['Сургутский муниципальный район', 'Сургут'] level_1
Профсоюз
[] level_0
Профсоюз
[] level_1


In [129]:
full_address_list

[[['628426 ханты-мансийский автономный округ югра г  сургут ул  профсоюзов д  45',
   'level_1',
   'Ханты-Мансийский',
   ['Ханты-Мансийский муниципальный район']],
  ['628426 ханты-мансийский автономный округ югра г  сургут ул  профсоюзов д  45',
   'level_1',
   'Сургут',
   ['Сургутский муниципальный район', 'Сургут']]]]

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

text = "Ханты-Мансийский автономный округ - Югра 628405, г Сургут, пр-кт Комсомольский, д.10/1"
tokens = word_tokenize(text)
stop_words = set(stopwords.words('russian'))
filtered_tokens = [word for word in tokens if word not in stop_words]

print(filtered_tokens)

['Ханты-Мансийский', 'автономный', 'округ', '-', 'Югра', '628405', ',', 'г', 'Сургут', ',', 'пр-кт', 'Комсомольский', ',', 'д.10/1']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")
text = "Лемматизированная форма слова листья это лист"
tokens = word_tokenize(text)
lemmatized_words = [stemmer.stem(word) for word in tokens]
print(lemmatized_words)

['лемматизирова', 'форм', 'слов', 'лист', 'эт', 'лист']


***

**СЭМД-РЭМД рабочий**

In [2]:
# загрузка моего справочника Роскадастра
rus_parh = r"C:\work\scripts\справочники\1.1.roskanastr_main.xlsx"
df_rus = pd.read_excel(rus_parh, sheet_name="clusters", index_col=None)

In [128]:
# загрузка СЭМД-РЭМД
rzn_path = r"C:\work\Медкнижки\semd_remd\sr\объекты_медосмотры_предварительные_для_МЗ_02_04_2024.xlsx"
s194_path = r"C:\work\Медкнижки\semd_remd\sr\РЭМД. 194 04.04.xlsx"
s230_path = r"C:\work\Медкнижки\semd_remd\sr\РЭМД. 230 04.04.xlsx"

df_rzn = pd.read_excel(rzn_path, sheet_name="rzn", index_col=None)
df_s194 = pd.read_excel(s194_path, sheet_name="194", index_col=None).fillna(0)
# df_s230 = pd.read_excel(s230_path, sheet_name="230", index_col=None).fillna(0)

In [5]:
def addr_parce(str):
    """парсим адрес, разделённый запятыми"""

    bad_list = ["пгт", "город", "село", "г.о.", "р. п."]
    
    lst = [x.strip() for x in str.replace(".", "").split(",") if len(x) <= 42]
    lst = [x for x in lst if re.sub(r"[0-9]+", "", x)]
    # lst = [x for x in lst if x not in bad_list]
    list = []
    for element in lst:
        element = " ".join([x for x in element.split(" ") if len(x) > 2])
        list.append(element)
    list_res = []
    for element in list:
        l_tmp = []
        l = element.split(" ")
        for e in l:
            if (len(e) < 7 and e.find("-") == 2) or e in bad_list:
                e = ""
            l_tmp.append(e)
        element = " ".join(l_tmp).strip().replace("  ", " ")
        list_res.append(element)
    return list_res[:4]

# старая функция
def find_id_old(str):
    """по адресу отыскивает id в справочнике Роскадастра
    для работы нужен df_rus"""

    tmp = addr_parce(str)

    rus_reg_list = set(df_rus["region"].tolist())
    rus_mun_list = set(df_rus["municipality"].tolist())
    rus_sett_list = set(df_rus["settelment"].tolist())

    rg = ""
    mn = ""
    st = ""
    for element in tmp:
        if element in rus_reg_list and rg == "":
            rg = element
        elif element in rus_mun_list and mn == "":
            mn = element
        elif element in rus_sett_list and st == "":
            st = element
        else:
            # print(f"не найдено: {element}")
            pass
    df_ru_0 = df_rus[(df_rus["region"] == rg) & (df_rus["municipality"] == mn)]
    df_ru_1 = df_rus[(df_rus["region"] == rg) & (df_rus["settelment"] == st)]
    df_ru = df_ru_0.copy() if df_ru_1.empty else df_ru_1.copy()
    df_r = df_ru[["id", "region", "municipality", "type", "settelment"]]
    if df_r.shape[0] == 1:
        # res = str(df_r["region"].tolist()[0]) + " " + str(df_r["municipality"].tolist()[0]) + " " + str(df_r["type"].tolist()[0]) + " " + str(df_r["settelment"].tolist()[0])
        index = df_r["id"].values[0]
    else:
        res = "всё сложно"
        index = 999999
    return index


# новая функция
def find_id(str):
    """по адресу отыскивает id в справочнике Роскадастра
    для работы нужен df_rus"""

    tmp = addr_parce(str)

    rus_reg_list = set(df_rus["region"].tolist())
    rus_mun_list = set(df_rus["municipality"].tolist())
    rus_sett_list = set(df_rus["settelment"].tolist())

    rus_reg_dict = {}
    rus_mun_dict = {}
    rus_sett_dict = {}

    for element in rus_reg_list:
        if type(element) == float or type(element) == int:
            element = f"{element}"
        key = element.replace(" ", "").replace("-", "").replace("ё", "е").lower()
        rus_reg_dict[key] = element

    for element in rus_mun_list:
        if type(element) == float or type(element) == int:
            element = f"{element}"
        key = element.replace(" ", "").replace("-", "").replace("ё", "е").lower()
        rus_mun_dict[key] = element

    for element in rus_sett_list:
        if type(element) == float or type(element) == int:
            element = f"{element}"
        key = element.replace(" ", "").replace("-", "").replace("ё", "е").lower()
        rus_sett_dict[key] = element

    rg = ""
    mn = ""
    st = ""
    for element in tmp:
        if type(element) == float or type(element) == int:
            element = f"{element}"
        element = element.replace(" ", "").replace("-", "").replace("ё", "е").lower()
        if element in rus_reg_dict and rg == "":
            rg = rus_reg_dict[element]
        elif element in rus_mun_dict and mn == "":
            mn = rus_mun_dict[element]
        elif element in rus_sett_dict and st == "":
            st = rus_sett_dict[element]
        else:
            pass
    df_ru_0 = df_rus[(df_rus["region"] == rg) & (df_rus["municipality"] == mn)]
    df_ru_1 = df_rus[(df_rus["region"] == rg) & (df_rus["settelment"] == st)]
    df_ru = df_ru_0.copy() if df_ru_1.empty else df_ru_1.copy()
    # df_ru = df_ru_0.copy() if df_ru_0.shape[0] < df_ru_1.shape[0] else df_ru_1.copy() # сомнительно!
    if df_ru.shape[0] > 1:
        df_ru = df_rus[(df_rus["region"] == rg) & (df_rus["municipality"] == mn)].head(1)
        # df_ru.iloc[0]
    df_r = df_ru[["id", "region", "municipality", "type", "settelment"]]
    if df_r.shape[0] == 1:
        index = df_r["id"].values[0]
    else:
        index = 0
    return index

In [4]:
a_0 = "628200, Ханты-Мансийский автономный округ - Югра, Кондинский, село Междуреченский, ул. Кондинская, д. 3"
a_1 = "628260, Ханты-Мансийский автономный округ -  Югра, г. Югорск, ул. Попова, д. 29"
a_2 = "677010, Республика Саха (Якутия), г. Якутск, шоссе Сергеляхское, д. 4"
a_3 = "677005, Республика Саха (Якутия), г. Якутск, ул. Петра Алексеева, д. 83 к. 18А, пом. 5-6"
a_4 = "650002, Кемеровская область - Кузбасс, г.о. Кемеровский, г. Кемерово, б-р имени академика Л.С. Барбараша, стр. 6"
a_5 = "650055, Кемеровская область - Кузбасс, г. Кемерово, пр-кт Кузнецкий, д. 103 а"
a_6 = "600015, Владимирская область, г.о. город Владимир, г. Владимир, ул. Разина, д. 51, нежилые помещения №1-20, 22-29, расположенные на 1-ом этаже; нежилые помещения №6-36, расположенные на 2-ом этаже согласно технического плана здания"
a_7 = "600000, Владимирская область, г. Владимир, ул. Малые Ременники, д. 11, нежилое помещение № I на 1-ом этаже"

In [17]:
addr_parce(a_1)

['Ханты-Мансийский автономный округ Югра', 'Югорск', 'Попова', '']

In [22]:
index = find_id(a_7)
index

12436

In [23]:
df_rus[df_rus["id"] == index]

,id,region,municipality,type,settelment,latitude,longitude,latitude_g,longitude_g,city_population,cluster,municipality2
12435,12436,Владимирская область,"Населённые пункты, не относящиеся к районам",город обл. подч.,Владимир,56°08.8',40°24.6',56.146667,40.41,346771.0,[100000;+∞],"Населённые пункты, не относящиеся к районам"


In [102]:
# str = a_2

# tmp = addr_parce(str)

# rus_reg_list = set(df_rus["region"].tolist())
# rus_mun_list = set(df_rus["municipality"].tolist())
# rus_sett_list = set(df_rus["settelment"].tolist())

# rus_reg_dict = {}
# rus_mun_dict = {}
# rus_sett_dict = {}

# for element in rus_reg_list:
#     if type(element) == float or type(element) == int:
#         element = f"{element}"
#     key = element.replace(" ", "").replace("-", "").replace("ё", "е").lower()
#     rus_reg_dict[key] = element

# for element in rus_mun_list:
#     if type(element) == float or type(element) == int:
#         element = f"{element}"
#     key = element.replace(" ", "").replace("-", "").replace("ё", "е").lower()
#     rus_mun_dict[key] = element

# for element in rus_sett_list:
#     if type(element) == float or type(element) == int:
#         element = f"{element}"
#     key = element.replace(" ", "").replace("-", "").replace("ё", "е").lower()
#     rus_sett_dict[key] = element

# rg = ""
# mn = ""
# st = ""
# for element in tmp:
#     print(element)
#     if type(element) == float or type(element) == int:
#         element = f"{element}"
#     element = element.replace(" ", "").replace("-", "").replace("ё", "е").lower()
#     print(element)
#     if element in rus_reg_dict and rg == "":
#         print(rus_reg_dict[element])
#         rg = rus_reg_dict[element]
#     elif element in rus_mun_dict and mn == "":
#         mn = rus_mun_dict[element]
#     elif element in rus_sett_dict and st == "":
#         st = rus_sett_dict[element]
#     else:
#         pass
# df_ru_0 = df_rus[(df_rus["region"] == rg) & (df_rus["municipality"] == mn)]
# df_ru_1 = df_rus[(df_rus["region"] == rg) & (df_rus["settelment"] == st)]
# df_ru = df_ru_0.copy() if df_ru_1.empty else df_ru_1.copy()
# # df_ru = df_ru_0.copy() if df_ru_0.shape[0] < df_ru_1.shape[0] else df_ru_1.copy() # сомнительно!
# if df_ru.shape[0] > 1:
#     df_ru = df_rus[(df_rus["region"] == rg) & (df_rus["municipality"] == mn)].head(1)
#     # df_ru.iloc[0]
# df_r = df_ru[["id", "region", "municipality", "type", "settelment"]]
# if df_r.shape[0] == 1:
#     index = df_r["id"].values[0]
# else:
#     index = 0
# print(index)

In [208]:
# стало функцией, но пока пусть будет, не удаляю
# tmp = addr_parce(str)

# rus_reg_list = set(df_rus["region"].tolist())
# rus_mun_list = set(df_rus["municipality"].tolist())
# rus_sett_list = set(df_rus["settelment"].tolist())

# rg = ""
# mn = ""
# st = ""
# for element in tmp:
#     if element in rus_reg_list and rg == "":
#         rg= element
#     elif element in rus_mun_list and mn == "":
#         mn = element
#     elif element in rus_sett_list and st == "":
#         st = element
#     else:
#         print(f"не найдено: {element}")
# df_ru_0 = df_rus[(df_rus["region"] == rg) & (df_rus["municipality"] == mn)]
# df_ru_1 = df_rus[(df_rus["region"] == rg) & (df_rus["settelment"] == st)]
# df_ru = df_ru_0.copy() if df_ru_1.empty else df_ru_1.copy()
# df_r = df_ru[["region", "municipality", "type", "settelment"]]
# if df_r.shape[0] == 1:
#     res = str(df_r["region"].tolist()[0]) + " " + str(df_r["municipality"].tolist()[0]) + " " + str(df_r["type"].tolist()[0]) + " " + str(df_r["settelment"].tolist()[0])
#     index = df_r.index
# else:
#     res = "всё сложно"
# print(index[0])
# print(res)

In [112]:
df_rzn["vpr"] = df_rzn["Адрес объекта"].apply(find_id)

In [130]:
df_s194["vpr"] = df_s194["Адрес объекта"].apply(find_id)

In [131]:
df_s194

,Субъект РФ,Наименование МО,OID МО,Наименование СП МО,OID СП МО,Адрес СП МО,Зарегистрировано ЭМД,Для ВПР,Количество МО,Адрес объекта,vpr
0,Российская Федерация,0,0,0,0,0,695302,0.0,4115.0,"Российская Федерация,",0
1,Алтайский край,0,0,0,0,0,20461,0.0,107.0,"Алтайский край,",0
2,Алтайский край,"КГБУЗ ""АЛЕЙСКАЯ ЦРБ""",1.2.643.5.1.13.13.12.2.22.1576,Поликлиника,1.2.643.5.1.13.13.12.2.22.1576.0.63516,"658130, г Алейск, ул им В.Олешко, д.30",1257,1257.0,0.0,"Алтайский край, 658130, г Алейск, ул им В.Олеш...",756
3,Алтайский край,"КГБУЗ ""АЛТАЙСКАЯ ЦРБ""",1.2.643.5.1.13.13.12.2.22.1735,Поликлиника,1.2.643.5.1.13.13.12.2.22.1735.0.81800,"659650, с Алтайское, ул К.Маркса, д.197",89,89.0,0.0,"Алтайский край, 659650, с Алтайское, ул К.Марк...",0
4,Алтайский край,"КГБУЗ ""АЛТАЙСКИЙ КРАЕВОЙ ГОСПИТАЛЬ ДЛЯ ВЕТЕРАН...",1.2.643.5.1.13.13.12.2.22.1666,консультативно-поликлиническое отделение,1.2.643.5.1.13.13.12.2.22.1666.0.53714,"656045, г Барнаул, тракт Змеиногорский, д.112",4,4.0,0.0,"Алтайский край, 656045, г Барнаул, тракт Змеин...",757
...,...,...,...,...,...,...,...,...,...,...,...
5639,Ярославская область,"ООО ""МЕДИЦИНСКИЙ ЦЕНТР ДИАГНОСТИКИ И ПРОФИЛАКТ...",1.2.643.5.1.13.13.12.2.76.12207,"Клиника ""НТ-Медицина""",1.2.643.5.1.13.13.12.2.76.12207.0.412173,", г Ярославль, ул Некрасова, д.60",1,1.0,0.0,"Ярославская область, , г Ярославль, ул Некрасо...",152960
5640,Ярославская область,"ООО ПЦ ""БУДЬ ЗДОРОВ!""",1.2.643.5.1.13.13.12.2.76.17054,Не указано,Не указано,Не указано,14,14.0,0.0,"Ярославская область, Не указано",0
5641,Ярославская область,"ООО ""ЦЕНТР СЕМЕЙНОЙ МЕДИЦИНЫ""",1.2.643.5.1.13.13.12.2.76.14369,"ООО ""Центр семейной медицины""",1.2.643.5.1.13.13.12.2.76.14369.0.384953,"150062, г Ярославль, ул 5-я Яковлевская, д.17",23,23.0,0.0,"Ярославская область, 150062, г Ярославль, ул 5...",152960
5642,Ярославская область,"ООО ""ЧАСТНАЯ ПОЛИКЛИНИКА+""",1.2.643.5.1.13.13.12.2.76.22193,Частная поликлиника плюс,1.2.643.5.1.13.13.12.2.76.22193.0.519984,"150040, г Ярославль, ул Некрасова, д.37а",16,16.0,0.0,"Ярославская область, 150040, г Ярославль, ул Н...",152960


In [116]:
s_path = r"C:/Users/Admin/Desktop/result_2024-04-12.csv"
df_rzn.to_csv(
            s_path,
            sep=";",
            encoding="utf-8-sig",
            index=False,
)

In [132]:
s_194_path = r"C:/Users/Admin/Desktop/194_2024-04-12.csv"
df_s194.to_csv(s_194_path,
            sep=";",
            encoding="utf-8-sig",
            index=True,)

In [114]:
df_rzn

,Наименование организации,Полное наименование организации,ИНН,ОГРН,Регион организации,Юридический адрес,Регион объекта,Адрес объекта,Тип населенного пункта,Код ФИАС,Тип объекта,№,Дата,Статус,Работы/услуги,Категория риска РОМ (фарм.розн.),Категория риска РОМ (фарм.опт),vpr
0,"""Медицинский центр Дар""","Общество с ограниченной ответственностью ""Меди...",8602310766,1238600005115,Ханты-Мансийский автономный округ - Югра,"628426, Россия, Ханты-Мансийский автономный ок...",Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",NaN,f60cb57e-5248-4b9e-bd1d-603c1ef1b475,NaN,Л041-01350-86/00741812,23.10.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,147189
1,МЦ СургутАвиаМед,Общество с ограниченной ответственностью Медиц...,8602309680,1238600001672,Ханты-Мансийский автономный округ - Югра,"628422, Россия, Ханты-Мансийский автономный ок...",Ханты-Мансийский автономный округ - Югра,"628426, Ханты-Мансийский автономный округ - Юг...",NaN,6c2aadb8-4412-4ed5-9082-bd1131a3945b,NaN,Л041-01350-86/00666954,28.07.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,147189
2,"""МЕДСПРАВКА""","Общество с ограниченной ответственностью ""МЕДС...",2634110663,1222600010005,Ставропольский край,"355017, Россия, Ставропольский край, городской...",Ставропольский край,"355017, Ставропольский край, г Ставрополь, ул ...",NaN,0fc8937c-bb24-4a4f-b4ab-7cd0cd101747,NaN,Л041-01197-26/00639943,03.02.2023,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,127053
3,"""Предгорная районная больница""",государственное бюджетное учреждение здравоохр...,2618010703,1022600962240,Ставропольский край,"357350, Россия, Ставропольский край, Предгорны...",Ставропольский край,"357350, Ставропольский край, Предгорный р-н, с...",станица,58e93788-5cd4-4962-b334-378ffae40f96,NaN,Л041-01197-26/00574633,24.12.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,127168
4,"""Благодарненская районная больница""",государственное бюджетное учреждение здравоохр...,2605010625,1022602421863,Ставропольский край,"356420, Россия, Ставропольский край, Благодарн...",Ставропольский край,"356420, Ставропольский край, Благодарненский р...",NaN,b277c385-a390-4ac4-a85d-0162fee81c99,NaN,Л041-01197-26/00574277,28.01.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,126645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12177,УфИЦ РАН,Федеральное государственное бюджетное научное ...,274064870,1030204207582,Республика Башкортостан,"450098, Россия, Республика Башкортостан, Уфа, ...",Республика Башкортостан,"450054, Республика Башкортостан, г. Уфа, пр. О...",город,faf12493-d842-40bc-968a-938e86dd8d70,NaN,Л041-00110-02/00340505,01.11.2018,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,97695
12178,федеральное казенное учреждение здравоохранени...,федеральное казенное учреждение здравоохранени...,275006624,1020202776109,Республика Башкортостан,"450015, Россия, Республика Башкортостан, г. Уф...",Республика Башкортостан,"450015, Республика Башкортостан, г. Уфа, ул. К...",город,236bd30d-7d8d-4baa-a322-44806a79b34a,NaN,Л041-00110-02/00323564,02.12.2019,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,97695
12179,ФЕДЕРАЛЬНОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИ...,Федеральное казенное учреждение здравоохранен...,105047430,1060105011361,Республика Адыгея,"385006, Россия, Республика Адыгея, г. Майкоп, ...",Республика Адыгея,"385006, Республика Адыгея, г. Майкоп, ул. Стро...",город,105aa593-0e4b-4769-8c09-ab9ff4cb12fe,NaN,Л041-00110-01/00571612,12.12.2017,Действует,Медицинская деятельность: Приказ 866н; 1000. П...,NaN,NaN,93974
12180,"Фгбоу во ""Мгту"", мгту, майкопский государствен...",Федеральное государственное бюджетное образова...,105014177,1020100698595,Республика Адыгея,"385000, Россия, Республика Адыгея, г. Майкоп, ...",Республика Адыгея,"385000, Республика Адыгея, г. Майкоп, Комсомол...",город,da529930